# Goals and Overview

# Project

## Initialization

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

## Reading Data

In [ ]:
data = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0])

In [ ]:
data.sort_index(inplace=True)

In [ ]:
data

## Data Preparation

In [ ]:
data = data.resample('1H').sum()
data

In [ ]:
def make_features(data):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour

In [ ]:
make_features(data)

## Data Analysis

In [ ]:
data['num_orders'].resample('M').sum().plot()

plt.title('Number of Monthly Orders')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.legend()

plt.show()

In [ ]:
data['num_orders'].resample('M').sum()

When grouped by month, there is a clear upward trend in the number of orders over the six-month period. Each subsequent month shows an increase in the total number of orders compared to the previous month.

August 2018 saw the highest number of orders, with a total of 94,973 orders. This is more than double the number of orders in March 2018, indicating a significant increase in demand or activity.

In [ ]:
plt.figure(figsize=(10, 6))

data['num_orders']['2018-03-01'].plot(label='2018-03-01')
data['num_orders']['2018-03-02'].plot(label='2018-03-02')
data['num_orders']['2018-03-03'].plot(label='2018-03-03')

plt.title('Number of Orders for March 01-03')
plt.xlabel('Time')
plt.ylabel('Number of Orders')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

data['num_orders']['2018-05-01'].plot(label='2018-05-01')
data['num_orders']['2018-05-02'].plot(label='2018-05-02')
data['num_orders']['2018-05-03'].plot(label='2018-05-03')

plt.title('Number of Orders for May 01-03')
plt.xlabel('Time')
plt.ylabel('Number of Orders')
plt.legend()

plt.show()

Orders consistently start relatively high at midnight and gradually decrease until the early morning hours. There are notable peaks in orders between 8 AM to 11 AM. Order numbers fluctuate throughout the afternoon and early evening hours, with occasional peaks and dips.

In [ ]:
data[data['hour'] == 0]['num_orders']

In [ ]:
plt.figure(figsize=(10, 6))

data[data['hour'] == 0]['num_orders']['2018-03'].plot()

plt.title('Number of Orders at 12AM for 2018-03')
plt.xlabel('Day')
plt.ylabel('Number of Orders')

plt.show()

When analyzing the number of orders placed at 12AM in the month of March, we see a noticable dip on March 06. We will analyze that day below.

In [ ]:
plt.figure(figsize=(10, 6))

data['num_orders']['2018-03-06'].plot()

plt.title('Number of Orders for 2018-03-06')
plt.xlabel('Hour')
plt.ylabel('Number of Orders')

plt.show()

Despite the low amount of orders for 12AM on March 06th, the trends are consistent with the other days.

In [ ]:
plt.figure(figsize=(10, 6))

data[data['hour'] == 0]['num_orders']['2018-04'].plot()

plt.title('Number of Orders at 12AM for 2018-04')
plt.xlabel('Day')
plt.ylabel('Number of Orders')

plt.show()

In [ ]:
decomposed = seasonal_decompose(data['num_orders']['2018-03-01':'2018-03-07'])

plt.figure(figsize=(8, 8))

plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')

plt.tight_layout()

When analyzing the first week of March, despite fluctuations in trends, there is a consistent pattern in a 24H cycle.

In [ ]:
decomposed = seasonal_decompose(data['num_orders']['2018-04-01':'2018-04-29'])

plt.figure(figsize=(8, 8))

plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')

plt.tight_layout()

Looking at the month of April, we see similar results.

In [ ]:
decomposed = seasonal_decompose(data['num_orders'])

plt.figure(figsize=(8, 8))

plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')

plt.tight_layout()

Based on the analysis, there seems to be seasonality to a 24 hour day, while the number of orders trends up as months go by.

## Testing Models

In [ ]:
train, test = train_test_split(data, shuffle=False, test_size=0.10)

In [ ]:
df_stationarityTest = adfuller(train['num_orders'], autolag='AIC')
print("P-value: ", df_stationarityTest[1])

In [ ]:
print('Median hourly number of orders:', train['num_orders'].median())

pred_previous = test['num_orders'].shift()
pred_previous.iloc[0] = train['num_orders'].iloc[-1]
print('RMSE:', np.sqrt(mean_squared_error(test['num_orders'], pred_previous)))

### AR Model

In [ ]:
best_lag = None
use_seasonality = None
best_ar_rmse = float('inf')
best_ar_pred = None

lags = [10, 50, 100]
seasonality = [True, False]

for lag in lags:
    for seasonal in seasonality:
        
        mod = ar_select_order(endog=train['num_orders'], maxlag=lag)
        ar_order = mod.ar_lags

        ar_model = AutoReg(train['num_orders'], lags=ar_order, seasonal=seasonal)
        ar_model = ar_model.fit()

        start_value = len(train['num_orders'])
        end_value = len(train['num_orders']) + len(test['num_orders']) - 1
        ar_pred = ar_model.predict(start=start_value, end=end_value, dynamic=False)

        ar_rmse = np.sqrt(mean_squared_error(test['num_orders'], ar_pred))
        print(f'RMSE: {ar_rmse}')
        
        if ar_rmse < best_ar_rmse:
            best_lag = ar_order
            use_seasonality = seasonal
            best_ar_rmse = ar_rmse
            best_ar_pred = ar_pred
            
print('Optimal Lags:', best_lag)
print('Uses seasonal:', use_seasonality)
print('Best AR RMSE:', best_ar_rmse)

In [ ]:
plt.plot(best_ar_pred, color='blue', label='pred')
plt.plot(test['num_orders'], color='red', label='test')
plt.legend(loc="upper left")
plt.xticks(rotation=90)
plt.show()

### MA Model

In [ ]:
max_ma_order_range = [10, 20, 30]

best_ma_rmse = float('inf')
best_ma_order = None

for max_ma_order in max_ma_order_range:
    res = arma_order_select_ic(y=train['num_orders'], max_ar=0, max_ma=max_ma_order)
    ma_order = res.bic_min_order[1]
    
    ma_model = ARIMA(train['num_orders'], order=(0, 0, ma_order))
    ma_model = ma_model.fit()
    
    start_value = len(train['num_orders'])
    end_value = len(train['num_orders']) + len(test['num_orders']) - 1
    ma_pred = ma_model.predict(start=start_value, end=end_value, dynamic=False)
    
    rmse = np.sqrt(mean_squared_error(test['num_orders'], ma_pred))
    print(f'Max MA Order: {max_ma_order}, MA Order: {ma_order}, RMSE: {rmse}')
    
    if rmse < best_ma_rmse:
        best_ma_rmse = rmse
        best_ma_order = ma_order

print(f'Best RMSE: {best_ma_rmse}')
print(f'Best MA Order: {best_ma_order}')

### ARMA/ARIMA Model

In [ ]:
data = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0])
data = data.resample('1H').sum()
train, test = train_test_split(data, shuffle=False, test_size=0.10)

ar_order_range = [1, 2, 3]
diff_order_range = [0, 1]
ma_order_range = [1, 2, 3]

best_arima_rmse = float('inf')
best_ar_order = None
best_diff_order = None
best_ma_order = None

tscv = TimeSeriesSplit(n_splits=5)

for ar_order in ar_order_range:
    for diff_order in diff_order_range:
        for ma_order in ma_order_range:
            try:
                cv_rmse_list = []

                for train_index, val_index in tscv.split(train['num_orders']):
                    train_cv, val_cv = train.iloc[train_index], train.iloc[val_index]

                    arima_model = ARIMA(train_cv, order=(ar_order, diff_order, ma_order))
                    arima_model_fit = arima_model.fit()

                    start_value = len(train_cv)
                    end_value = len(train_cv) + len(val_cv) - 1
                    arima_pred = arima_model_fit.predict(start=start_value, end=end_value, dynamic=False)

                    rmse = np.sqrt(mean_squared_error(val_cv, arima_pred))
                    cv_rmse_list.append(rmse)

                avg_rmse = np.mean(cv_rmse_list)
                print(f'AR Order: {ar_order}, Diff Order: {diff_order}, MA Order: {ma_order}, Avg CV RMSE: {avg_rmse}')

                if avg_rmse < best_arima_rmse:
                    best_arima_rmse = avg_rmse
                    best_ar_order = ar_order
                    best_diff_order = diff_order
                    best_ma_order = ma_order

            except Exception as e:
                print(f'Error with parameters AR: {ar_order}, I: {diff_order}, MA: {ma_order} - {e}')

print(f'Best RMSE: {best_arima_rmse}')
print(f'Best AR Order: {best_ar_order}')
print(f'Best Diff Order: {best_diff_order}')
print(f'Best MA Order: {best_ma_order}')

best_arima_model = ARIMA(train['num_orders'], order=(best_ar_order, best_diff_order, best_ma_order))
best_arima_model_fit = best_arima_model.fit()

start_value = len(train['num_orders'])
end_value = len(train['num_orders']) + len(test['num_orders']) - 1
test_predictions = best_arima_model_fit.predict(start=start_value, end=end_value, dynamic=False)

arima_rmse = np.sqrt(mean_squared_error(test['num_orders'], test_predictions))
print(f'Test RMSE: {arima_rmse}')

### SARIMA MODEL

In [ ]:
sarima_model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
sarima_model_fit = sarima_model.fit(disp=False)

start_value = len(train['num_orders'])
end_value = len(train['num_orders']) + len(test['num_orders']) - 1
sarima_pred = sarima_model_fit.predict(start=start_value, end=end_value, dynamic=False)

plt.plot(test.index, sarima_pred, color='blue', label='Predictions')
plt.plot(test.index, test, color='red', label='Actual')
plt.legend(loc="upper left")
plt.xticks(rotation=90)
plt.show()

sarima_rmse = np.sqrt(mean_squared_error(test, sarima_pred))
print(f'RMSE: {rmse}')

## Testing

In [ ]:
best_ar_rmse

In [ ]:
best_ma_rmse

In [ ]:
arima_rmse

In [ ]:
sarima_rmse

## Conclusion

When comparing the various models with different hyperparameters, the SARIMA Model stands out as the most accurate with an RMSE of 44.

The AR model had a decent score, at 48.33, however missed the target RMSE.

The ARIMA Model had similar results, with a high training RMSE of 36, but when tested reached 64.